# MINI Interview Agent

A streamlined agent for conducting MINI (Mini International Neuropsychiatric Interview) assessments with intelligent response analysis.

## Setup and Dependencies

In [146]:
%pip install langchain openai tiktoken langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Core Implementation

In [147]:
import os
import getpass
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

def get_key():
    """Get OpenAI API key from .env file, environment, or user input."""
    # Clear any cached API key
    if 'OPENAI_API_KEY' in os.environ:
        del os.environ['OPENAI_API_KEY']
    
    # Load environment variables from .env file (force reload)
    load_dotenv(override=True)
    
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        api_key = getpass.getpass('Enter your OpenAI API key: ')
        os.environ['OPENAI_API_KEY'] = api_key
    
    # Verify key format
    if api_key and not api_key.startswith('sk-'):
        print("⚠️  Warning: API key doesn't start with 'sk-'")
    
    print(f"✅ API key loaded (ends with: ...{api_key[-10:] if api_key else 'None'})")
    return api_key

# Set up API key
get_key()
print("API key configured")

✅ API key loaded (ends with: ...cmKxGpLUIA)
API key configured


In [148]:
from langchain.agents import tool
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

@tool
def ask_patient(question: str) -> str:
    """Ask the patient the current MINI question and get their response."""
    print(f"Clinician: {question}")
    patient_response = input("Patient: ")
    print(f"Patient response: {patient_response}")  # Print patient answer
    return patient_response

def analyze_response(response: str, question_context: str) -> str:
    """Use LLM to analyze and classify patient's response as 'yes', 'no', or 'unclear' based on clinical context."""
    
    # Pre-filter common unclear responses to avoid LLM misclassification
    unclear_patterns = [
        "i don't know", "i dont know", "idk", "not sure", "maybe", "i'm not sure",
        "i am not sure", "uncertain", "hard to say", "depends", "sometimes",
        "kind of", "sort of", "partially", "it varies", "i think so but",
        "i'm confused", "i am confused", "what do you mean", "can you repeat"
    ]
    
    response_lower = response.lower().strip()
    
    # Check for explicit unclear responses first
    for pattern in unclear_patterns:
        if pattern in response_lower:
            return 'unclear'
    
    # If not obviously unclear, use LLM analysis
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    
    analysis_prompt = PromptTemplate(
        input_variables=["response", "question_context"],
        template="""You are a skilled clinician analyzing a patient's response to a MINI interview question.

Question context: {question_context}
Patient response: "{response}"

Analyze this response and classify it as exactly one of: 'yes', 'no', or 'unclear'

CRITICAL GUIDELINES:
- 'yes': Patient clearly and definitively indicates affirmative (agreement, presence of symptoms, positive response)
- 'no': Patient clearly and definitively indicates negative (disagreement, absence of symptoms, negative response)  
- 'unclear': ANY ambiguous, uncertain, contradictory, or hesitant response that needs clarification

IMPORTANT: If there is ANY doubt about the patient's intent, classify as 'unclear'. Better to ask for clarification than to misclassify.

Examples of UNCLEAR responses:
- "I don't know", "Maybe", "Sometimes", "It depends", "Not sure", "Kind of", "Sort of"
- Any response with uncertainty markers like "I think", "I guess", "Probably"
- Contradictory statements or requests for clarification

Respond with ONLY the classification word: yes, no, or unclear"""
    )
    
    try:
        result = analysis_prompt.format(response=response, question_context=question_context)
        llm_response = llm.invoke(result)
        
        # Extract and clean the classification
        classification = llm_response.content.strip().lower()
        
        # More strict classification logic
        if classification == 'yes':
            return 'yes'
        elif classification == 'no':
            return 'no'
        else:
            # Default to unclear if LLM gives anything other than clear yes/no
            return 'unclear'
            
    except Exception as e:
        print(f"Error in LLM analysis: {e}")
        return 'unclear'

@tool
def analyze_response_tool(response: str, question_context: str) -> str:
    """Tool version of analyze_response for agent use."""
    return analyze_response(response, question_context)

def ask_clarification(original_question: str, unclear_response: str) -> str:
    """Ask the patient to clarify their unclear response with an LLM-generated follow-up question."""
    
    # Use LLM to generate a contextually appropriate clarification question
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    clarification_prompt = PromptTemplate(
        input_variables=["original_question", "unclear_response"],
        template="""You are a skilled clinician conducting a MINI interview. The patient gave an unclear response that needs clarification.

Original question: "{original_question}"
Patient's unclear response: "{unclear_response}"

Generate a professional, empathetic clarification question that:
1. Acknowledges the patient's response respectfully
2. Gently asks for a clearer yes/no answer
3. May rephrase the original question if helpful
4. Uses natural, conversational language
5. Shows clinical empathy and understanding

The goal is to help the patient provide a clear yes or no response while maintaining rapport.

Clarification question:"""
    )
        # Generate the clarification question using LLM
    result = clarification_prompt.format(
        original_question=original_question, 
        unclear_response=unclear_response
    )
    llm_response = llm.invoke(result)
    clarification_question = llm_response.content.strip()
        
    # Present the LLM-generated clarification question
    print(f"Clinician: {clarification_question}")
    patient_response = input("Patient: ")
    print(f"Patient response: {patient_response}")
    return patient_response

@tool
def ask_clarification_tool(original_question: str, unclear_response: str) -> str:
    """Tool version of ask_clarification for agent use."""
    return ask_clarification(original_question, unclear_response)

@tool
def explain(current_question: str) -> str:
    """Clarify or rephrase the current question to help the patient understand."""
    explanations = {
        "anxiety": "Anxiety means feeling worried, nervous, or uneasy about something.",
        "panic": "Panic symptoms include rapid heartbeat, sweating, trembling, or feeling like you can't breathe.",
        "agoraphobia": "This refers to fear of being in places where escape might be difficult or help unavailable.",
        "avoidance": "Avoidance means staying away from situations that make you uncomfortable."
    }
    
    explanation = f"Let me clarify: {current_question}\n\n"
    for key, value in explanations.items():
        if key in current_question.lower():
            explanation += f"Note: {value}\n"
    
    return explanation

@tool
def end_module() -> str:
    """Signal that the module has ended either due to logic or patient response."""
    return "Module assessment complete. Thank you for your responses."

@tool
def get_next_question(current_question_id: str, patient_answer: str, question_context: str) -> str:
    """Determine the next question based on current question and patient's LLM-analyzed answer."""
    # Check if questions dictionary exists
    try:
        if current_question_id not in questions:
            return "END_MODULE"
        
        branching = questions[current_question_id].get("next", {})
        classified_answer = analyze_response(patient_answer, question_context)  # Uses the regular function
        
        next_q = branching.get(classified_answer, "END_MODULE")
        
        if next_q == "END_MODULE":
            return "END_MODULE"
        elif next_q in questions:
            return questions[next_q]["prompt"]
        else:
            return "END_MODULE"
    except NameError:
        # If questions is not defined yet, return a placeholder
        print("Warning: questions dictionary not loaded yet")
        return "QUESTIONS_NOT_LOADED"

# TODO: This is not working as expected. Need to fix it. For now we are going to use a simple yes/no criteria
# to determine diagnosis, not an LLM tool.
# @tool
# def evaluate_diagnosis_from_log(conversation_log, criteria_rules, module_name):
    """
    Evaluate diagnosis based on conversation log and diagnostic rules.

    Parameters:
        conversation_log (list of dicts): Each dict has question_id, classification, etc.
        criteria_rules (dict): e.g., {'agoraphobia': {'current': "E6 == 'YES'"}}
        module_name (str): e.g., "agoraphobia"

    Returns:
        str: "yes" or "no"
    """

    # Step 1: Extract relevant patient answers for the target module
    patient_responses = {}
    for entry in conversation_log:
        qid = entry.get("question_id")
        classification = entry.get("classification")
        if qid and classification:
            patient_responses[qid] = classification.strip().upper()

    # Step 2: Get the rule for the specified diagnosis
    rule = criteria_rules.get(module_name.lower(), {}).get('current', None)
    if not rule:
        raise ValueError(f"No criteria rule found for module '{module_name}'.")

    # Step 3: Prepare inputs for the LLM prompt
    prompt = PromptTemplate(
        input_variables=["response", "criteria"],
        template=(
            "You are a psychiatric diagnostic evaluator.\n"
            "Based on the patient's responses:\n\n"
            "{response}\n\n"
            "And the diagnostic rule:\n"
            "{criteria}\n\n"
            "Does the patient meet the criteria for this disorder?\n"
            "Answer only with 'Yes' or 'No'."
        )
    )

    formatted_response = "\n".join(f"{k}: {v}" for k, v in patient_responses.items())
    final_prompt = prompt.format(response=formatted_response, criteria=rule)

    # Step 4: Send prompt to LLM and normalize output
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    classification_response = llm([HumanMessage(content=final_prompt)])
    classification = classification_response.content.strip().lower()

    if "yes" in classification:
        return True
    else:
        return False

# List of tools for the agent (clean and focused)
tools = [ask_patient, analyze_response_tool, ask_clarification_tool, explain, end_module, get_next_question]

print("Core interview tools defined successfully!")

Core interview tools defined successfully!


## 3 Load MINI Questions (Module E)

In [149]:
# Load module data
import json, pprint

PATH = "mini_modules/working/module_e.json" 
with open(PATH, "r", encoding="utf-8") as f:
    module_data = json.load(f)

mini_script = module_data["questions"]     
module_info = module_data["module"] 
module_diagnostic_criteria = module_data["diagnosticCriteria"]

# Prepare questions dictionary for easy access
questions = {q["id"]: q for q in mini_script}

print(f"Loaded {len(mini_script)} questions for Module {module_info['id']}: {module_info['name']}\n")

for i, q in enumerate(mini_script[:2], start=1):
    print(f"{i}. {q['id']}: {q['prompt']}\n")

# Branching logic retrieval function
def show_branches(qid: str):
    if qid not in questions:
        raise KeyError(f"{qid!r} not found")
    next = questions[qid].get("next", {})
    print(f"Branching for {qid}:")
    pprint.pp(next)

# Example: inspect the first question’s branches
show_branches("E1")

Loaded 6 questions for Module E: Agoraphobia

1. E1: Do you feel anxious or uneasy in places or situations where help might not be available or escape difficult if you had a panic-like or embarrassing symptom, such as being in a crowd or queue, in an open space or crossing a bridge, in an enclosed space, when alone away from home, when alone at home, or traveling in a bus, train, car, or using public transportation?

2. E2: Do these situations almost always bring on fear or anxiety?

Branching for E1:
{'yes': 'E2', 'no': 'END_MODULE'}


## 4 Prompt Template

In [150]:
QUESTION_PROMPT = PromptTemplate(
    input_variables=["history", "question"],
    template="""You are an AI agent with the role of a skilled clinician trained in professional physchological diagnosis and care conducting an interview based on the MINI questionnaire. Be concise, empathetic, and professional.

Previous conversation:
{history}

Next question to ask:
{question}

Present this question in a natural, conversational way. Keep your response brief and focused, ask the question prompting a yes or no response from the patient."""
)

## 5 Class to Manage Interview State

In [151]:
# Interview State Management
import datetime

class InterviewState:
    def __init__(self):
        self.current_question_id = "E1"
        self.conversation_log = []
        self.is_complete = False
        self.interview_start_time = None
        self.is_diagnosis = None
    
    def reset(self):
        self.current_question_id = "E1"
        self.conversation_log = []
        self.is_complete = False
        self.interview_start_time = datetime.datetime.now()
        self.is_diagnosis = None

# Initialize global state
interview_state = InterviewState()

# Core tools for the interview agent (keeping the essential ones)
print("Interview tools ready: ask_patient, analyze_response, ask_clarification, explain")

Interview tools ready: ask_patient, analyze_response, ask_clarification, explain


## 6 Determine the Diagnosis Based on Interview

In [152]:
def determine_diagnosis(conversation_log):
    """Determine the diagnosis based on the patient interview.
    For now we are going to use a simple yes/no criteria to determine diagnosis.
    Later we will use diagnostic criteria from the module to determine the diagnosis.
    
    Returns:
        bool: True if all answers are 'yes', False if any answer is 'no' or 'unclear'
    """
    
    if not conversation_log:
        return False
        
    for entry in conversation_log:
        classification = entry.get('classification', '').lower()
        if classification != 'yes':
            return False
        
    return True

## 7 Export the Interview as a CSV Report

In [153]:
import os
import csv
import datetime


def export_results_to_csv():
    """Export the interview results to a CSV file."""    
    reports_dir = "reports"
    if not os.path.exists(reports_dir):
        os.makedirs(reports_dir)        
    
    # Use timestamp to name the file, guaranteed to be unique
    if interview_state.interview_start_time:
        timestamp = interview_state.interview_start_time.strftime("%Y%m%d_%H%M%S")
    else:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    
    filename = f"mini_interview_{timestamp}.csv"
    filepath = os.path.join(reports_dir, filename)
        
    with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Question ID', 'Question', 'Patient Response', 'Classification', 'Clarifications Needed']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for entry in interview_state.conversation_log:
            writer.writerow({
                'Question ID': entry['question_id'],
                'Question': entry['question'],
                'Patient Response': entry['patient_response'],
                'Classification': entry['classification'],
                'Clarifications Needed': entry.get('clarifications_needed', 0)
            })
    
    print(f"\nResults exported to: {filepath}")    

In [154]:
# Simplified Agent-Based Interview Workflow
def run_interview_with_agent():
    """Run the MINI interview with agent assistance for response analysis and clarification."""
    # Reset interview state
    interview_state.reset()
    
    print(f"Starting MINI Module {module_info['id']}: {module_info['name']}")
    print("Using AI Agent for intelligent response analysis and clarification")
    print("=" * 70)
    
    # Initialize LLM for agent decisions
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    
    while not interview_state.is_complete:
        # Get current question
        if interview_state.current_question_id == "END_MODULE":
            break
            
        if interview_state.current_question_id not in questions:
            print("No more questions available.")
            break
            
        current_question = questions[interview_state.current_question_id]
        
        # Display question
        print(f"\n[Question {interview_state.current_question_id}]")
        print(f"Clinician: {current_question['prompt']}")
        
        # Get patient response
        patient_response = input("Patient: ")
        print(f"Patient answered: {patient_response}")
        
        # Use agent to analyze response
        classification = analyze_response(patient_response, current_question['prompt'])
        print(f"LLM Analysis: {classification}")
        
        # If unclear, use agent for clarification
        max_clarifications = 2
        clarification_count = 0
        
        while classification == 'unclear' and clarification_count < max_clarifications:
            clarification_count += 1
            print(f"\n[Clarification {clarification_count}/{max_clarifications}]")
            
            # Get clarified response
            clarified_response = ask_clarification(current_question['prompt'], patient_response)
            print(f"Clarified response: {clarified_response}")
            
            # Re-analyze clarified response
            classification = analyze_response(clarified_response, current_question['prompt'])
            print(f"LLM Analysis of clarification: {classification}")
            
            # Update patient response to the clarified version
            patient_response = clarified_response
        
        # If still unclear after max attempts, default to 'no'
        if classification == 'unclear':
            print(f"Still unclear after {max_clarifications} attempts. Defaulting to 'no' to continue.")
            classification = 'no'
        
        # Log the interaction
        interview_state.conversation_log.append({
            "question_id": interview_state.current_question_id,
            "question": current_question['prompt'],
            "patient_response": patient_response,
            "classification": classification,
            "clarifications_needed": clarification_count
        })
        
        # Use branching logic to determine next question
        branching = current_question.get("next", {})
        next_question_id = branching.get(classification, "END_MODULE")
        
        print(f"Based on '{classification}' response, next: {next_question_id}")
        
        if next_question_id == "END_MODULE":
            interview_state.is_complete = True
            print("Interview complete based on branching logic.")
        else:
            interview_state.current_question_id = next_question_id
    
    # Print final summary
    print("\n" + "=" * 70)
    print("INTERVIEW COMPLETE")
    print("=" * 70)
    
    print(f"\nModule: {module_info['name']}")
    print(f"Total questions asked: {len(interview_state.conversation_log)}")
    
    print("\nCONVERSATION SUMMARY:")
    print("-" * 50)
    
    for i, entry in enumerate(interview_state.conversation_log, 1):
        print(f"\n{i}. Question {entry['question_id']}:")
        print(f"   Clinician: {entry['question']}")
        print(f"   Patient: {entry['patient_response']}")
        print(f"   Classification: {entry['classification']}")
        if entry.get('clarifications_needed', 0) > 0:
            print(f"   Clarifications needed: {entry['clarifications_needed']}")
    
    interview_state.is_diagnosis = determine_diagnosis(interview_state.conversation_log)

    diagnosis_text = "Positive" if interview_state.is_diagnosis else "Negative"
    print(f"Diagnosis result for Agoraphobia module: {diagnosis_text}")
    
    # Export results of the interview to CSV
    export_results_to_csv()
    
    return interview_state.conversation_log

# Demo function for testing LLM analysis
def run_llm_analysis_demo():
    """Demonstrate the LLM-powered analysis capabilities."""
    print("=== LLM Analysis Tool Demonstration ===")
    
    test_cases = [
        ("yes definitely", "Do you feel anxious in crowded places?"),
        ("not really, maybe sometimes", "Do you avoid certain situations?"),
        ("I get nervous but I still go", "Do these situations cause you distress?"),
        ("absolutely not", "Have you experienced panic attacks?"),
        ("well, it depends on the day", "Do you feel this way consistently?")
    ]
    
    print("\nTesting LLM-powered response analysis:")
    for response, question in test_cases:
        try:
            classification = analyze_response(response, question)
            print(f"'{response}' -> {classification}")
        except Exception as e:
            print(f"Error analyzing '{response}': {e}")
    
    print("\nLLM analysis tool working correctly!")

## 8 Demo Run with Agent Tools

In [ ]:
# Execute the interview (using the agent-based approach)
results = run_interview_with_agent()

Starting MINI Module E: Agoraphobia
Using AI Agent for intelligent response analysis and clarification

[Question E1]
Clinician: Do you feel anxious or uneasy in places or situations where help might not be available or escape difficult if you had a panic-like or embarrassing symptom, such as being in a crowd or queue, in an open space or crossing a bridge, in an enclosed space, when alone away from home, when alone at home, or traveling in a bus, train, car, or using public transportation?
Patient answered: theres a cat climbing up the tree
LLM Analysis: unclear

[Clarification 1/2]
Clinician: "I appreciate you sharing that with me; it sounds like you’re noticing something interesting. Just to make sure I understand your feelings better, can you tell me if you ever feel anxious or uneasy in situations where you might not have help available, like being in a crowd or traveling alone? A simple 'yes' or 'no' would really help me understand your experience."
